In [1]:
import netgen.gui
%gui tk
from netgen.geom2d import *
from ngsolve import *



In [2]:
geo = SplineGeometry()
geo.AddRectangle((0,0), (2,2),
                 bcs=["b","r","t","l"],
                 leftdomain=1)
geo.AddRectangle((1,0.9), (1.3,1.4),
                 bcs=["b2","r2","t2","l2"],
                 leftdomain=2, rightdomain=1)
geo.SetMaterial (1, "outer")
geo.SetMaterial (2, "inner")
mesh = Mesh(geo.GenerateMesh(maxh=0.1))

In [3]:
fes1 = H1(mesh, definedon="inner")

u1 = GridFunction(fes1, "u1")
u1.Set(x*y)
Draw(u1)


In [4]:
fes = H1(mesh, order=3, dirichlet="b|l|r")

u = fes.TrialFunction()
v = fes.TestFunction()
gfu = GridFunction(fes)

In [5]:
f = LinearForm(fes)
f += SymbolicLFI(u1*v, definedon=mesh.Materials("inner"))
f += SymbolicLFI(0.1*v, definedon=mesh.Boundaries("t"))
f.Assemble()
# mesh.Materials("inner") and mesh.Boundaries("t") are "region" objects

In [6]:
a = BilinearForm(fes)
a += SymbolicBFI(grad(u)*grad(v))
a.Assemble()

# Solve the problem:
gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
Draw (gfu)


In [7]:
mesh.GetMaterials()

('outer', 'inner')

In [8]:
mesh.GetBoundaries()

('b', 'r', 't', 'l', 'b2', 'r2', 't2', 'l2')

In [9]:
mesh.Materials("inner")

In [10]:
mesh.Boundaries("t")  

In [11]:
print(mesh.Materials("inner").Mask())
print(mesh.Materials("[a-z]*").Mask())  # can use regexp
print(mesh.Boundaries('t|l').Mask())



0: 01
0: 11
0: 00110000


In [12]:
io = mesh.Materials("inner") + mesh.Materials("outer")
print(io.Mask()) # add regions

0: 11


In [13]:
c = ~mesh.Materials("inner") # complement of a region
print(c.Mask())

0: 10


In [14]:
diff = mesh.Materials("inner|outer") - mesh.Materials("outer")
print(diff.Mask()) # difference of two regions


0: 01


Set a piecewise constant CoefficientiFunction using the subdomains:

In [15]:
domain_values = {'inner': 3.7,  'outer': 1}
values_list = [domain_values[mat]
               for mat in mesh.GetMaterials()]
cf = CoefficientFunction(values_list)
Draw(cf, mesh, 'piecewise')
# so if you initialize a coefficient function with a list, it assumes that each element is for a domain

In [16]:
values_list

[1, 3.7]

In [18]:
cf = CoefficientFunction([x*y, x-y])
Draw(cf, mesh, 'pw2')

In [29]:
cf = CoefficientFunction([2-x,y])
g = GridFunction(H1(mesh), name='bdry')
g.Set(cf, definedon=~mesh.Boundaries('')) # complement of empty set gives all boundaries
Draw(g, max=2, min=0)


In [25]:
c = ~mesh.Boundaries('')
print(c.Mask())

0: 11111111


In [20]:
g.Set(cf, definedon=mesh.Boundaries('b|r'))
Redraw()


In [21]:
g = GridFunction(H1(mesh))
g.Set(cf, BND)
Draw(g, max=2, min=0)


In [22]:
g = GridFunction(H1(mesh, dirichlet='b|r'))
g.Set(cf, BND)
Draw(g, max=2, min=0)